### 아이디어
- 13925(수열과 쿼리 13)과 비슷하게 구간 min과 max를 둘다 사용하는 lazy propagation을 사용할 것 같다.
- remove연산은 min, add연산은 max를 사용한다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, idx, val) :
    self.seg[idx] = self.fl(self.seg[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.seg[idx] = self.f(self.seg[idx*2], self.seg[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol() :
  N, K = map(int, input().split())
  f = max
  fl = lambda i, v, cnt: min(i, v[1]) if v[1] else max(i, v[1])
  fll = lambda x, y: (0, max(x[1], y[1])) if y[1] else (1, min(x[1], y[1]))
  lst = LST([0] * N, f, fl, fll, ldefault=(2, 9999))

  for _ in range(K) :
    q, s, e, x = map(int, input().split())
    if q == 1 :
      lst.update(s, e, (1, x))
    elif q == 2 :
      lst.update(s, e, (0, x))
  
  print(*lst.seg[N:])

sol()